In [1]:
#Checking directory 
%pwd

'/home/hearth/ML/course/deeplearning1/nbs'

In [2]:
import os, sys
import numpy as np
#reference to imp directories
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data/statefarm/'

In [3]:
import importlib
import utils; importlib.reload(utils)
from utils import *


%matplotlib inline

Using cuDNN version 7003 on context None
Mapped name None to device cuda0: GeForce GTX 1080 Ti (0000:01:00.0)
Using Theano backend.


Create Validation and Sample sets and the appropriate directories

In [4]:
import vgg16; importlib.reload(vgg16)
from vgg16 import Vgg16
vgg = Vgg16()

/home/hearth/ML/course/deeplearning1/nbs/vgg16_bn.h5


In [5]:
%cd $DATA_HOME_DIR
path = DATA_HOME_DIR+'/'

/home/hearth/ML/course/deeplearning1/nbs/data/statefarm


## Creating directory structure

In [ ]:
%mkdir valid
%mkdir results
%mkdir -p sample/train
%mkdir -p sample/test
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p test/unknown

## Creating Validation set based on drivers. 
Need to have separate drivers in validation and train set to prevent overfitting based on drivers. Drivers aren't important but their actions are.

In [6]:
for i in range(0,10):
    os.mkdir(valid_path+'c'+str(i))

NameError: name 'valid_path' is not defined

In [ ]:
drivers = df.groupby('subject').groups.keys()
print (drivers)

In [ ]:
import pandas as pd
df= pd.read_csv('driver_imgs_list.csv')
drivers = df.groupby('subject').groups.keys()
print (drivers)

In [ ]:
driver = df.loc[df['subject'] == 'p049']
for index, row in driver.iterrows():
    location = row[1]+'/'
    name = row[2]
    fname = train_path+location+name
    if (os.path.isfile(fname)):
        os.rename(train_path+location+name, valid_path+location+name)

## Setting folder locations

In [5]:
%cd $DATA_HOME_DIR/

/home/hearth/ML/course/deeplearning1/nbs/data/statefarm


In [6]:
from shutil import copyfile
test_path = path + '/test/' #We use all the test data
results_path=DATA_HOME_DIR + '/results/'
train_path=path + 'train/'
valid_path=path + 'valid/'
fulltrain = DATA_HOME_DIR+'/train/'
fulltest = DATA_HOME_DIR+'/test/'
sample_path = path+'/sample/train/'

## Moving data to appropriate directories

In [ ]:
%cd $DATA_HOME_DIR/train

In [ ]:
from glob import glob
paths = glob('*/')
print (paths)

Making validation set and all other sets. Switch between copyfile and os.rename as appropriate

In [ ]:
for i in paths:
    tpath = train_path+i
    npath = sample_path+i
    %mkdir $npath
    %cd $tpath
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(1000): copyfile(shuf[i], npath+shuf[i])


Creating sample test

In [ ]:
%cd $fulltest
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(30): copyfile(shuf[i], test_path+shuf[i])

In [ ]:
# Create single 'unknown' class for test set
%cd $DATA_HOME_DIR/test
%mv *.jpg unknown/

## The above are initialization steps

In [6]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired

test_path = DATA_HOME_DIR + '/test/' #We use all the test data
results_path=DATA_HOME_DIR + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'

/home/hearth/ML/course/deeplearning1/nbs/data/statefarm


In [7]:
batch_size = 64
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size)
test_batches = vgg.get_batches(path+'test', batch_size=batch_size)

Found 18509 images belonging to 10 classes.
Found 3915 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


## 2Conv model

In [ ]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, steps_per_epoch=int(np.ceil(batches.samples/64)), nb_epoch=1, validation_data=val_batches, 
                     validation_steps=int(np.ceil(val_batches.samples/64)))
    #model.optimizer.lr = 0.001
    #model.fit_generator(batches, steps_per_epoch=int(np.ceil(batches.samples/64)), nb_epoch=1, validation_data=val_batches, 
     #                validation_steps=int(np.ceil(val_batches.samples/64)))
    return model

In [ ]:
model = conv1(batches)

In [ ]:
def test(self, path, batch_size=8):
        print(path)
        test_batches = get_batches(self, path, shuffle=False, batch_size=batch_size, class_mode=None)
        print (test_batches.samples)
        return test_batches, self.model.predict_generator(test_batches, int(np.ceil(test_batches.samples/batch_size)))
def get_batches(self, path, gen=image.ImageDataGenerator(), shuffle=False, batch_size=8, class_mode='categorical'):
        return gen.flow_from_directory(path, target_size=(224,224),
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [ ]:
batches, preds = test(bn_model, test_path, batch_size = batch_size*2)

In [ ]:
batches, preds = vgg.test(test_path, batch_size = batch_size*2)

In [ ]:
save_array(results_path+'test_preds.dat', preds)
save_array(results_path+'filenames.dat', batches.filenames)

## VGG model

In [8]:
model = vgg.model
# Here, [-1] means that, fetch the last element from the array returned by the for loop
last_conv_idx = [i for i, l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [9]:
conv_model = Sequential(conv_layers)

In [10]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 18509 images belonging to 10 classes.
Found 3915 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [11]:
%cd $results_path
rm -R conv*.dat
%cd $path

/home/hearth/ML/course/deeplearning1/nbs/data/statefarm/results
/home/hearth/ML/course/deeplearning1/nbs/data/statefarm


In [15]:
#conv_feat2 = conv_model.predict_generator(batches, np.ceil(batches.samples/batch_size))
conv_val_feat2 = conv_model.predict_generator(val_batches, np.ceil(val_batches.samples/batch_size))
#conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)

PreCalculating convolution features for train, val and test data to speed up computations. The precalculated features are stored in bcolz array and written to disk to free up RAM. 

In [11]:
fname = results_path+'conv_featFinal.dat'
for i in range(batches.n // batch_size+1):
    conv_feat = conv_model.predict_on_batch(batches.next()[0])
    if not i:
        c = bcolz.carray(conv_feat, rootdir=fname, mode='a')
    else:
        c.append(conv_feat)
    c.shape
c.flush()

In [12]:
fname = results_path+'conv_val_featFinal.dat'
for i in range(val_batches.n // batch_size+1):
    conv_val_feat = conv_model.predict_on_batch(val_batches.next()[0])
    if not i:
        c = bcolz.carray(conv_val_feat, rootdir=fname, mode='a')
    else:
        c.append(conv_val_feat)
c.flush()

In [13]:
fname = results_path+'conv_test_featFinal.dat'
for i in range(test_batches.n // batch_size+1):
    conv_test_feat = conv_model.predict_on_batch(test_batches.next()[0])
    if not i:
        c = bcolz.carray(conv_test_feat, rootdir=fname, mode='a')
    else:
        c.append(conv_test_feat)
c.flush()

In [ ]:
#conv_val_feat = load_array(path+'results/conv_val_feat.dat')
#conv_val_feat2 = load_array(path+'results/conv_val_y.dat')

In [14]:
fname = results_path+'conv_val_yFinal.dat'
save_array(fname, val_labels)
fname = results_path+'conv_train_yFinal.dat'
save_array(fname, trn_labels)

Post pre-calculation

In [15]:
X=bcolz.open(results_path+'conv_featFinal.dat', mode='r')
y= bcolz.open(results_path+'conv_train_yFinal.dat', mode='r')

In [16]:
valX=bcolz.open(results_path+'conv_val_featFinal.dat', mode='r')
valy= bcolz.open(results_path+'conv_val_yFinal.dat', mode='r')

In [17]:
trn_batches=BcolzArrayIterator(X,y, batch_size=batch_size, shuffle=True)
vl_batches=BcolzArrayIterator(valX,valy, batch_size=batch_size, shuffle=True)

In [18]:
testX=bcolz.open(path+'results/conv_test_featFinal.dat', mode='r')

In [19]:
tst_batches=BcolzArrayIterator(testX, batch_size=batch_size, shuffle=False)

In [69]:
preds = bn_model.predict(conv_val_feat, batch_size=batch_size*2)

## Dense layers on pretrained conv layers
Edit: VGG with Batchnorm using batchnorm-vgg trained model and weights. http://files.fast.ai/models/

Train: 18509
Validate: 3915
Test:  79726

In [20]:
trn_batches.N

18509

In [21]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [22]:
p=0.8
#print (trn_batches)

In [23]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
##bn_model.fit(conv_feat2, trn_labels, batch_size=batch_size, nb_epoch=10, 
  ##           validation_data=(conv_val_feat2, val_labels))

In [25]:
bn_model.fit_generator(trn_batches, nb_epoch=10, steps_per_epoch=np.ceil(trn_batches.N/batch_size), validation_data=vl_batches, 
                     validation_steps=int(np.ceil(vl_batches.N/batch_size)))

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<utils.Bco..., steps_per_epoch=290.0, validation_data=<utils.Bco..., validation_steps=62, epochs=10)`
  


Epoch 1/10
290/290 [==============================] - 19s - loss: 1.6574 - acc: 0.5515 - val_loss: 0.5802 - val_acc: 0.8097
Epoch 2/10
290/290 [==============================] - 5s - loss: 0.3183 - acc: 0.9004 - val_loss: 0.5459 - val_acc: 0.8322
Epoch 3/10
290/290 [==============================] - 5s - loss: 0.1556 - acc: 0.9560 - val_loss: 0.4264 - val_acc: 0.8662
Epoch 4/10
290/290 [==============================] - 5s - loss: 0.0997 - acc: 0.9726 - val_loss: 0.4577 - val_acc: 0.8720
Epoch 5/10
290/290 [==============================] - 5s - loss: 0.0700 - acc: 0.9812 - val_loss: 0.4321 - val_acc: 0.8544
Epoch 6/10
290/290 [==============================] - 5s - loss: 0.0531 - acc: 0.9863 - val_loss: 0.4162 - val_acc: 0.8639
Epoch 7/10
290/290 [==============================] - 5s - loss: 0.0458 - acc: 0.9877 - val_loss: 0.4810 - val_acc: 0.8447
Epoch 8/10
290/290 [==============================] - 5s - loss: 0.0483 - acc: 0.9865 - val_loss: 0.4169 - val_acc: 0.8815
Epoch 9/10
290/

In [26]:
bn_model.optimizer.lr=0.01
bn_model.fit_generator(trn_batches, nb_epoch=10, steps_per_epoch=np.ceil(trn_batches.N/batch_size), validation_data=vl_batches, 
                     validation_steps=int(np.ceil(vl_batches.N/batch_size)))

Epoch 1/10
  9/290 [..............................] - ETA: 4s - loss: 0.0253 - acc: 0.9948

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<utils.Bco..., steps_per_epoch=290.0, validation_data=<utils.Bco..., validation_steps=62, epochs=10)`
  This is separate from the ipykernel package so we can avoid doing imports until


290/290 [==============================] - 5s - loss: 0.0437 - acc: 0.9866 - val_loss: 0.6854 - val_acc: 0.8245
Epoch 2/10
290/290 [==============================] - 5s - loss: 0.0361 - acc: 0.9895 - val_loss: 0.7715 - val_acc: 0.8247
Epoch 3/10
290/290 [==============================] - 5s - loss: 0.0281 - acc: 0.9919 - val_loss: 0.5185 - val_acc: 0.8606
Epoch 4/10
290/290 [==============================] - 5s - loss: 0.0254 - acc: 0.9922 - val_loss: 0.7761 - val_acc: 0.8291
Epoch 5/10
290/290 [==============================] - 5s - loss: 0.0272 - acc: 0.9923 - val_loss: 0.4671 - val_acc: 0.8733
Epoch 6/10
290/290 [==============================] - 5s - loss: 0.0270 - acc: 0.9919 - val_loss: 0.8083 - val_acc: 0.8301
Epoch 7/10
290/290 [==============================] - 5s - loss: 0.0256 - acc: 0.9927 - val_loss: 0.7623 - val_acc: 0.8365
Epoch 8/10
290/290 [==============================] - 5s - loss: 0.0217 - acc: 0.9938 - val_loss: 0.7086 - val_acc: 0.8181
Epoch 9/10
290/290 [=======

In [27]:
bn_model.save_weights(path+'models/conv8_1Final.h5')

## Pseudo Labelling

In [28]:
test_pseudo = bn_model.predict_generator(tst_batches, steps=np.ceil(tst_batches.N/batch_size), verbose=1)

1243/1246 [============================>.] - ETA: 0s

In [75]:
np.set_printoptions(suppress=True)
test_pseudo[3]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.], dtype=float32)

In [29]:
comb_pseudo = np.concatenate([trn_labels, test_pseudo])

In [30]:
X1=bcolz.open(path+'results/conv_featFinal.dat', mode='r')
trn_batches=BcolzArrayIterator(X1, batch_size=batch_size, shuffle=False)
trn_batches.N

18509

In [31]:
X2=bcolz.open(path+'results/conv_test_featFinal.dat', mode='r')
tst_batches=BcolzArrayIterator(X2, batch_size=batch_size, shuffle=False)
tst_batches.N

79726

In [32]:
fname = path+'results/conv_feat_extFinal.dat'
batch = trn_batches.next()
c = bcolz.carray(batch, rootdir=fname, mode='w')
for i in range(trn_batches.N // batch_size):
    batch = trn_batches.next()
    c.append(batch)
for i in range(tst_batches.N // batch_size+1):
    batch = tst_batches.next()
    c.append(batch)
c.flush()

In [33]:
X3=bcolz.open(path+'results/conv_feat_extFinal.dat', mode='r')

X3.shape

(98235, 512, 14, 14)

In [34]:
fname = path+'results/conv_ptest_yFinal.dat'
save_array(fname, comb_pseudo)

In [35]:
comb_pseudo.shape

(98235, 10)

Load pre calculated features and train on train+test set

In [36]:
pX=bcolz.open(path+'results/conv_feat_extFinal.dat', mode='r')
py= bcolz.open(path+'results/conv_ptest_yFinal.dat', mode='r')

In [37]:
tstComb_batches=BcolzArrayIterator(pX, py, batch_size=batch_size, shuffle=True)

In [38]:
bn_model.load_weights(path+'models/conv8_1Final.h5')

In [39]:
bn_model.optimizer.lr=0.001
bn_model.fit_generator(tstComb_batches, nb_epoch=10, steps_per_epoch=np.ceil(tstComb_batches.N/batch_size), validation_data=vl_batches, 
                     validation_steps=int(np.ceil(vl_batches.N/batch_size)))

Epoch 1/10
   5/1535 [..............................] - ETA: 51s - loss: 1.2713 - acc: 0.7906

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<utils.Bco..., steps_per_epoch=1535.0, validation_data=<utils.Bco..., validation_steps=62, epochs=10)`
  This is separate from the ipykernel package so we can avoid doing imports until


1535/1535 [==============================] - 76s - loss: 0.5719 - acc: 0.8569 - val_loss: 0.3865 - val_acc: 0.8771
Epoch 2/10
1535/1535 [==============================] - 32s - loss: 0.4419 - acc: 0.8912 - val_loss: 0.3790 - val_acc: 0.8932
Epoch 3/10
1535/1535 [==============================] - 26s - loss: 0.4205 - acc: 0.8975 - val_loss: 0.4173 - val_acc: 0.8748
Epoch 4/10
1535/1535 [==============================] - 26s - loss: 0.4081 - acc: 0.9023 - val_loss: 0.3956 - val_acc: 0.8600
Epoch 5/10
1535/1535 [==============================] - 27s - loss: 0.3963 - acc: 0.9071 - val_loss: 0.3398 - val_acc: 0.8912
Epoch 6/10
1535/1535 [==============================] - 27s - loss: 0.3882 - acc: 0.9083 - val_loss: 0.4123 - val_acc: 0.8718
Epoch 7/10
1535/1535 [==============================] - 26s - loss: 0.3810 - acc: 0.9118 - val_loss: 0.3861 - val_acc: 0.8866
Epoch 8/10
1535/1535 [==============================] - 26s - loss: 0.3749 - acc: 0.9139 - val_loss: 0.3807 - val_acc: 0.8713
Epo

In [41]:
bn_model.optimizer.lr=0.01
bn_model.fit_generator(tstComb_batches, nb_epoch=10, steps_per_epoch=np.ceil(tstComb_batches.N/batch_size), validation_data=vl_batches, 
                     validation_steps=int(np.ceil(vl_batches.N/batch_size)))

Epoch 1/10
   8/1535 [..............................] - ETA: 28s - loss: 0.3279 - acc: 0.9336

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<utils.Bco..., steps_per_epoch=1535.0, validation_data=<utils.Bco..., validation_steps=62, epochs=10)`
  This is separate from the ipykernel package so we can avoid doing imports until


1535/1535 [==============================] - 26s - loss: 0.3439 - acc: 0.9251 - val_loss: 0.3923 - val_acc: 0.8841
Epoch 2/10
1535/1535 [==============================] - 26s - loss: 0.3439 - acc: 0.9249 - val_loss: 0.3955 - val_acc: 0.8689
Epoch 3/10
1535/1535 [==============================] - 26s - loss: 0.3429 - acc: 0.9260 - val_loss: 0.4050 - val_acc: 0.8674
Epoch 4/10
1535/1535 [==============================] - 26s - loss: 0.3410 - acc: 0.9261 - val_loss: 0.3645 - val_acc: 0.8807
Epoch 5/10
1535/1535 [==============================] - 26s - loss: 0.3412 - acc: 0.9268 - val_loss: 0.4082 - val_acc: 0.8705
Epoch 6/10
1535/1535 [==============================] - 26s - loss: 0.3383 - acc: 0.9284 - val_loss: 0.3822 - val_acc: 0.8874
Epoch 7/10
1535/1535 [==============================] - 26s - loss: 0.3378 - acc: 0.9291 - val_loss: 0.3906 - val_acc: 0.8769
Epoch 8/10
1535/1535 [==============================] - 26s - loss: 0.3370 - acc: 0.9279 - val_loss: 0.3867 - val_acc: 0.8848
Epo

In [42]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [43]:
keras.metrics.categorical_crossentropy(val_labels, do_clip(preds, 0.9050)).eval()

NameError: name 'preds' is not defined

In [71]:
pX=bcolz.open(path+'results/conv_test_feat.dat', mode='r')
tst_batches=BcolzArrayIterator(pX, batch_size=batch_size, shuffle=False)

In [44]:
test_pred = bn_model.predict_generator(tst_batches, steps=np.ceil(tst_batches.N/batch_size), verbose=1)

1245/1246 [============================>.] - ETA: 0s

In [45]:
subm = do_clip(test_pred,0.8840)

In [46]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)
subm_name = path+'results/subm2.gz'

In [47]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'img', [a[8:] for a in test_filenames])
submission.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_1.jpg,0.012889,0.012889,0.012889,0.012889,0.012889,0.884000,0.012889,0.012889,0.012889,0.012889
1,img_10.jpg,0.012889,0.012889,0.012889,0.012889,0.012889,0.884000,0.012889,0.012889,0.012889,0.012889
2,img_100.jpg,0.884000,0.012889,0.012889,0.012889,0.012889,0.012889,0.012889,0.012889,0.012889,0.012889
3,img_1000.jpg,0.012889,0.012889,0.012889,0.012889,0.012889,0.012889,0.012889,0.012889,0.884000,0.012889
4,img_100000.jpg,0.012889,0.012889,0.012889,0.884000,0.012889,0.012889,0.012889,0.012889,0.012889,0.012889


In [48]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [49]:
FileLink(subm_name)

/home/hearth/ML/course/deeplearning1/nbs/data/statefarm/results/subm2.gz

## Making submissions, old method

In [ ]:
filenames = val_batches.filenames
expected_labels = val_batches.classes #0 or 1

#Round our predictions to 0/1 to generate labels
print (filenames[1], probs[1])

In [ ]:
preds = load_array(results_path + 'test_preds.dat')
filenames = load_array(results_path + 'filenames.dat')

In [ ]:
batches = vgg.get_batches(test_path, batch_size=batch_size)

In [ ]:
import re
filenames = batches.filenames
a = []

for idx, val in enumerate(filenames):
    a.append(val[8:])
    

In [ ]:
classes = sorted(val_batches.class_indices, key=val_batches.class_indices.get)
submission = pd.DataFrame(preds, columns=classes)

In [ ]:
print (submission)
submission['img']=a

In [ ]:
cols = submission.columns.tolist()
cols = cols[-1:] + cols[:-1]
submission= submission[cols]

In [ ]:
submission.to_csv('firstSubVGG', index=False)


In [ ]:
submission.shape

In [ ]:
preds = load_array('/home/hearth/ML/course/courses/deeplearning1/nbs/data/dogscats/' + 'test_preds.dat')
filenames = load_array('/home/hearth/ML/course/courses/deeplearning1/nbs/data/dogscats/' + 'filenames.dat')